In [2]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point

In [24]:
#get city
city = ox.geocode_to_gdf("Helsinki, Finland")

#load data
hex_grid = gpd.read_parquet("../data/processed/hex_grid.parquet")

In [6]:
#get road networks
roads = ox.features_from_place("Helsinki, Finland", tags={"highway": True}).to_crs(hex_grid.crs)

#keep onlyn roads (linestring geometries)
roads = roads[roads.geom_type.isin(["LineString", "MultiLineString"])]

In [ ]:
#reproject CRS
crs_projected = "EPSG:3067"
hex_grid_proj = hex_grid.to_crs(crs_projected)
roads_proj = roads.to_crs(crs_projected)

#clip roads to hexes and measure length
hex_grid_proj["road_density"] = hex_grid_proj.geometry.apply(
    lambda hex: roads_proj.clip(hex).length.sum())

In [ ]:

hospitals = ox.features_from_place(
    "Helsinki, Finland",
    tags={"amenity": "hospital"}
)
hospitals = hospitals[~hospitals.geometry.is_empty]
hospitals_proj = hospitals.to_crs("EPSG:3067")

hex_grid_proj['centroid'] = hex_grid_proj.geometry.centroid

def count_hospitals_near(point, hospitals_gdf, radius_m=1000):
    buffer = point.buffer(radius_m)
    return hospitals_gdf.geometry.intersects(buffer).sum()

#apply funtion to each hex centroid
hex_grid_proj['hospital_count'] = hex_grid_proj['centroid'].apply(lambda pt: count_hospitals_near(pt, hospitals_proj))

In [ ]:

city_proj = city.to_crs("EPSG:3067")
city_center = city_proj.geometry.centroid.iloc[0]

hex_grid_proj['dist_to_center'] = hex_grid_proj['centroid'].distance(city_center)


In [ ]:
hex_grid_proj.to_parquet("../data/processed/hex_features.parquet")